In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Eq, Le, Ge

In [2]:
assets = ['b', 's'] # buying and selling assets
base_symbols = ['s', 'l', 'w', 'j', 'e', 'Delta', 'a'] 
# spot price, liquidity, weight, jump size, exponent (positive offset from min/max), delta, anchor price
all_symbols = {}
for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        if base == 'e':
            temp_dict[f"{base}_{asset}"] = symbols(f"{base}_{asset}", nonnegative=True, integer=True)
        else:
            temp_dict[f"{base}_{asset}"] = symbols(f"{base}_{asset}", positive=True, integer=True)
    all_symbols[asset] = tuple(temp_dict.values())

all_symbols

{'b': (s_b, l_b, w_b, j_b, e_b, Delta_b, a_b),
 's': (s_s, l_s, w_s, j_s, e_s, Delta_s, a_s)}

In [3]:
e_b = all_symbols["b"][base_symbols.index("e")];
e_s = all_symbols["s"][base_symbols.index("e")];
Delta_b = all_symbols["b"][base_symbols.index("Delta")];
Delta_s = all_symbols["s"][base_symbols.index("Delta")];

In [4]:
def deltaBySpot_(s, l, w, j, e, Delta, a):
  return simplify((s - l * w) / w)

deltaBySpot = {asset: deltaBySpot_(*all_symbols[asset]) for asset in assets}
deltaBySpot['b'] = -deltaBySpot['b']
deltaBySpot['b']

l_b - s_b/w_b

In [5]:
deltaBySpot["s"];

-l_s + s_s/w_s

In [6]:
def expBound_(asset, s, l, w, j, e, Delta, a):
    return simplify(log(w * l /a, 1 + 1/j))

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b)))

In [7]:
def spotByExp_(asset, s, l, w, j, e, Delta, a):
  if asset == 'b':
    e_ = expBound[asset] - e
  else:
    e_ = expBound[asset] + e
  return simplify(a * ((1 + 1/j) ** e_)
)
spotByExp = {asset: spotByExp_(asset, *all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))

In [8]:
spotByExp["s"];

a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))

In [9]:
def deltaByExp_(asset, s, l, w, j, e, Delta, a):
    return simplify(deltaBySpot[asset].subs(s, spotByExp[asset]))

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b

In [10]:
deltaByExp["s"];

a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s

In [11]:
def deltaLimit_(asset, s, l, w, j, e, Delta, a):
    e = all_symbols[asset][base_symbols.index('e')]
    return simplify(solveset(deltaByExp[asset] >= 0, e, domain=S.Integers).condition)

deltaLimit = {asset: deltaLimit_(asset, *all_symbols[asset]) for asset in assets}
deltaLimit['b']

a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b - l_b <= 0

In [12]:
deltaLimit["s"];

a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s >= 0

In [13]:
solve(Eq(deltaLimit["b"].lhs, 0), e_b);

[0]

In [14]:
solve(Eq(deltaLimit["s"].lhs, 0), e_s);

[0]

Those two imply we gain nothing from deltaLimit

In [15]:
eff = Delta_s / Delta_b;
eff;

Delta_s/Delta_b

In [16]:
deltaByExp_b = deltaByExp["b"];
deltaByExp_s = deltaByExp["s"];

effByExps = eff.subs({ Delta_b: deltaByExp_b, Delta_s: deltaByExp_s });
effByExps;

(a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s)/(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)

In [17]:
H = hessian(effByExps, (e_b, e_s));
H;

Matrix([
[2*a_b**2*(1 + 1/j_b)**(-2*e_b + 2*log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))*(a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s)*log(1 + 1/j_b)**2/(w_b**2*(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)**3) + a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))*(a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s)*log(1 + 1/j_b)**2/(w_b*(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)**2), -a_b*a_s*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))*log(1 + 1/j_b)*log(1 + 1/j_s)/(w_b*w_s*(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)**2)],
[                                                                                                                                                                                                 

In [18]:
# checking for convexity. Both need to be nonnegative
topLeft = H[0,0] #simplify(H[0, 0])
#det = simplify(H.det())

topLeft

2*a_b**2*(1 + 1/j_b)**(-2*e_b + 2*log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))*(a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s)*log(1 + 1/j_b)**2/(w_b**2*(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)**3) + a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))*(a_s*(1 + 1/j_s)**(e_s + log((l_s*w_s/a_s)**(1/log((j_s + 1)/j_s))))/w_s - l_s)*log(1 + 1/j_b)**2/(w_b*(-a_b*(1 + 1/j_b)**(-e_b + log((l_b*w_b/a_b)**(1/log((j_b + 1)/j_b))))/w_b + l_b)**2)

In [19]:
topLeft_ = simplify(topLeft);
topLeft_;

KeyboardInterrupt: 

In [ ]:
nonneg_topLeft_eb_ = solveset(topLeft_ >= 0, e_b, domain = S.Integers);
nonneg_topLeft_eb_;

In [ ]:
nonneg_topLeft_eb = solveset(topLeft >= 0, e_b, domain = S.Integers);
nonneg_topLeft_eb;

In [ ]:
nonneg_topLeft_es_ = solveset(topLeft_ >= 0, e_s, domain = S.Integers);
nonneg_topLeft_es_;

In [ ]:
nonneg_topLeft_es = solveset(topLeft >= 0, e_s, domain = S.Integers);
nonneg_topLeft_es;

In [ ]:
det = H.det();
det;

In [ ]:
det = simplify(det);
det;

In [ ]:
nonneg_det_eb = solveset(det >= 0, e_b, domain = S.Integers);
nonneg_det_eb;

In [ ]:
nonneg_det_es = solveset(det >= 0, e_s, domain = S.Integers);
nonneg_det_es;

In [ ]:
nonneg_topLeft = simplify(nonneg_topLeft_eb.condition);
nonneg_topLeft;

In [ ]:
simplify(nonneg_topLeft_eb.condition.lhs - nonneg_topLeft_es.condition.lhs);

In [ ]:
nonneg_det = simplify(nonneg_det_eb.condition);
nonneg_det;

In [ ]:
simplify(nonneg_det_eb.condition.lhs - nonneg_det_es.condition.lhs);